In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='ticks')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/lish-moa/train_features.csv')
df_test = pd.read_csv('../input/lish-moa/test_features.csv')
df_train_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
df_train_unscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')

print(df_train.head())
print(df_train_scored.head())

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
data_train=pd.concat([df_train,df_test])
data_train.head()


In [ ]:
print(data_train.columns)
print(df_train_scored.columns)

In [ ]:
data_train.describe()

In [ ]:
df_train_scored.describe()

In [ ]:
print('Shape of Train Data: ', data_train.shape)
print('Shape of Train Scored Data: ', df_train_scored.shape)
print('Shape of Train Unscored Data: ', df_train_unscored.shape)

In [ ]:
round((data_train.isnull().sum()/len(data_train))*100,2)

In [ ]:
data_train.columns

In [ ]:
data_train['cp_type'].describe()

In [ ]:
sns.countplot(data_train['cp_type'])
plt.title('Train: Treatment Doses: Low and High',weight='bold', fontsize=18)

In [ ]:
sns.countplot(data_train['cp_dose'])
plt.title('Train: Control and treated samples', fontsize=15, weight='bold')

In [ ]:
data_train['cp_type'].value_counts()

In [ ]:
data_train.cp_type.value_counts(normalize=True).plot(kind='pie', figsize=(12, 5), fontsize=12,
                                                         title='CP Type', autopct='%1.1f%%')
plt.show()

In [ ]:
#Datasets for treated and control experiments
treated= data_train[data_train['cp_type']=='trt_cp']
control= data_train[data_train['cp_type']=='ctl_vehicle']

In [ ]:
data_train.cp_time.value_counts(normalize=True).plot(kind='bar', figsize=(12, 5), fontsize=14,
                                                         title='CP Time', xlabel='Time')
plt.show()

In [ ]:
treated

In [ ]:
data_train.cp_dose.value_counts(normalize=True).plot(kind='bar', figsize=(12, 5), fontsize=14, 
                                                         title='CP Dose', xlabel='Dose')
plt.show()

In [ ]:
#Treatment time datasets
cp24= data_train[data_train['cp_time']== 24]
cp48= data_train[data_train['cp_time']== 48]
cp72= data_train[data_train['cp_time']== 72]

In [ ]:
#Merge scored and nonscored labels
drugs= pd.merge(df_train_scored, df_train_unscored, on='sig_id', how='inner')
drugs

In [ ]:
#Treated drugs:
treated_list = treated['sig_id'].to_list()
drugs_tr= df_train_scored[df_train_scored['sig_id'].isin(treated_list)]

unscored= df_train_unscored[df_train_unscored['sig_id'].isin(treated_list)]
scored= df_train_scored[df_train_scored['sig_id'].isin(treated_list)]

In [ ]:
#adt= All Drugs Treated
adt= drugs[drugs['sig_id'].isin(treated_list)]
c_cols = [col for col in df_train.columns if 'c-' in col]
cells=treated[c_cols]

g_cols = [col for col in df_train.columns if 'g-' in col]
genes=treated[g_cols]

In [ ]:
g_cols = sns.pairplot(data_train[g_cols[:10]])
plt.show()

In [ ]:
g_cols = sns.pairplot(data_train[c_cols[:10]])
plt.show()

In [ ]:
def plotd(f1):
    plt.style.use('seaborn')
    sns.set_style('whitegrid')
    fig = plt.figure(figsize=(15,5))
    #1 rows 2 cols
    #first row, first col
    ax1 = plt.subplot2grid((1,2),(0,0))
    plt.hist(control[f1], bins=4, color='mediumpurple',alpha=0.5)
    plt.title(f'control: {f1}',weight='bold', fontsize=18)
    #first row sec col
    ax1 = plt.subplot2grid((1,2),(0,1))
    plt.hist(treated[f1], bins=4, color='darkcyan',alpha=0.5)
    plt.title(f'Treated with compound: {f1}',weight='bold', fontsize=18)
    plt.show()

In [ ]:
def plott(f1):
    sns.set_style('whitegrid')
    fig = plt.figure(figsize=(15,5))
    #1 rows 2 cols
    #first row, first col
    ax1 = plt.subplot2grid((1,3),(0,0))
    plt.hist(cp24[f1], bins=3, color='deepskyblue',alpha=0.5)
    plt.title(f'Treatment duration 24h: {f1}',weight='bold', fontsize=14)
    #first row sec col
    ax1 = plt.subplot2grid((1,3),(0,1))
    plt.hist(cp48[f1], bins=3, color='lightgreen',alpha=0.5)
    plt.title(f'Treatment duration 48h: {f1}',weight='bold', fontsize=14)
    #first row 3rd column
    ax1 = plt.subplot2grid((1,3),(0,2))
    plt.hist(cp72[f1], bins=3, color='gold',alpha=0.5)
    plt.title(f'Treatment duration 72h: {f1}',weight='bold', fontsize=14)
    plt.show()

In [ ]:
def plotf(f1, f2, f3, f4):
    sns.set_style('whitegrid')
    fig= plt.figure(figsize=(15,10))
    ax1 = plt.subplot2grid((2,2),(0,0))
    plt.hist(df_train[f1], bins=3, color='orange', alpha=0.7)
    plt.title(f1,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    #first row sec col
    ax1 = plt.subplot2grid((2,2), (0, 1))
    plt.hist(df_train[f2], bins=3, alpha=0.7)
    plt.title(f2,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    #Second row first column
    ax1 = plt.subplot2grid((2,2), (1, 0))
    plt.hist(df_train[f3], bins=3, color='red', alpha=0.7)
    plt.title(f3,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    #second row second column
    ax1 = plt.subplot2grid((2,2), (1, 1))
    plt.hist(df_train[f4], bins=3, color='green', alpha=0.7)
    plt.title(f4,weight='bold', fontsize=18)
    plt.yticks(weight='bold')
    plt.xticks(weight='bold')
    return plt.show()

In [ ]:
def ploth(data, w=15, h=9):
    plt.figure(figsize=(w,h))
    sns.heatmap(data.corr(), cmap='hot')
    plt.title('Correlation between the drugs', fontsize=18, weight='bold')
    return plt.show()

# corrs function: Show dataframe of high correlation between features
def corrs(data, col1='Gene 1', col2='Gene 2',rows=5,thresh=0.8, pos=[1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,31,33,35]):
        #Correlation between genes
        corre= data.corr()
         #Unstack the dataframe
        s = corre.unstack()
        so = s.sort_values(kind="quicksort", ascending=False)
        #Create new dataframe
        so2= pd.DataFrame(so).reset_index()
        so2= so2.rename(columns={0: 'correlation', 'level_0':col1, 'level_1': col2})
        #Filter out the coef 1 correlation between the same drugs
        so2= so2[so2['correlation'] != 1]
        #Drop pair duplicates
        so2= so2.reset_index()
        pos = pos
        so2= so2.drop(so2.index[pos])
        so2= so2.drop('index', axis=1)
        #Show the first 10 high correlations
        cm = sns.light_palette("Red", as_cmap=True)
        s = so2.head(rows).style.background_gradient(cmap=cm)
        print(f"{len(so2[so2['correlation']>thresh])/2} {col1} pairs have +{thresh} correlation.")
        return s

In [ ]:
sns.set_style('whitegrid')
fig = plt.figure(figsize=(15,5))
ax1 = plt.subplot2grid((1,2),(0,0))
sns.countplot(x='cp_type', data=df_train, palette='rainbow', alpha=0.75)
plt.title('Train: Control and treated samples', fontsize=15, weight='bold')
ax1 = plt.subplot2grid((1,2),(0,1))
sns.countplot(x='cp_dose', data=df_train, palette='Purples', alpha=0.75)
plt.title('Train: Treatment Doses: Low and High',weight='bold', fontsize=18)
plt.show()

In [ ]:
df_train_scored = df_train_scored.sum()[1:].sort_values()
df_train_scored[:50].plot(kind='barh', title='Least Target Occurances', fontsize='14', figsize=(5, 20))
plt.show()

In [ ]:
df_train_scored[-50:].plot(kind='barh', title='Most Target Occurences', fontsize='12', figsize=(12, 20))
plt.show()

In [ ]:
# We can see outliers in genomes.

plt.figure(figsize=(100, 100))
sns.heatmap(data_train[c_cols].corr())
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.distplot( data_train['cp_time'], color='green', bins=5)
plt.title("Train: Treatment Time ", fontsize=15)

In [ ]:
plotf('c-10', 'c-50', 'c-70', 'c-90')

In [ ]:
plott('c-30')

In [ ]:
c_cols = [col for col in df_train.columns if 'c-' in col]
#Filter the columns c-
cells=treated[c_cols]
#Plot heatmap FOR corelations
plt.figure(figsize=(12,6))
sns.heatmap(cells.corr(), cmap='coolwarm', alpha=0.9)
plt.title('Correlation: Cell viability', fontsize=15, weight='bold')
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:
gc_corr = {}
useful_col = c_cols - gc_corr.keys()
useful_col = list(useful_col)
useful_col = useful_col + ['cp_type', 'cp_dose', 'cp_time']

In [ ]:
useful_col

In [ ]:
len(useful_col), len(c_cols)

In [ ]:
corrs(cells, 'Cell', 'Cell 2', rows=15)

In [ ]:
corrs(genes, 'Gene', 'Gene 2')

In [ ]:
#Correlation between drugs
corre= genes.corr()
#Unstack the dataframe
s = corre.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
#Create new dataframe
so2= pd.DataFrame(so).reset_index()
so2= so2.rename(columns={0: 'correlation', 'level_0':'Drug 1', 'level_1': 'Drug2'})
#Filter out the coef 1 correlation between the same drugs
so2= so2[so2['correlation'] != 1]
#Drop pair duplicates
so2= so2.reset_index()
so2= so2.sort_values(by=['correlation'])
pos = [1,3,5,7,9,11,13,15,17,19,21]
so2= so2.drop(so2.index[pos])
so2= so2.round(decimals=4)
so2=so2.drop('index', axis=1)
so3=so2.head(4)
#Show the first 10 high correlations
cm = sns.light_palette("Red", as_cmap=True)
s = so2.head().style.background_gradient(cmap=cm)
s

In [ ]:
# scored= df_train_scored[df_train_scored['sig_id'].isin(treated_list)]

#Count unique values per column
cols = drugs_tr.columns.to_list() # specify the columns whose unique values you want here
uniques = {col: drugs_tr[col].nunique() for col in cols}
uniques=pd.DataFrame(uniques, index=[0]).T
uniques=uniques.rename(columns={0:'count'})
uniques= uniques.drop('sig_id', axis=0)

In [ ]:

#Calculate the mean values
average=scored.mean()
average=pd.DataFrame(average)
average=average.rename(columns={ 0: 'mean'})
average['percentage']= average['mean']*100
#Filter just the drugs with mean >0.01
average_filtered= average[average['mean'] > 0.01]
average_filtered= average_filtered.reset_index()
average_filtered= average_filtered.rename(columns={'index': 'drug'})

In [ ]:
ctl_vehicle_id = data_train[data_train.cp_type == 'ctl_vehicle']['sig_id']
ctl_vehicle_id = list(ctl_vehicle_id)

In [ ]:
# sum_target_cp_type_ctl_vehicle = data_train[df_train.sig_id.isin( ctl_vehicle_id)].sum()[1:].sum()

# print('Training Data - For cp_type - ctl_vehicle total sum of targets : {}'.format(sum_target_cp_type_ctl_vehicle))

In [ ]:
data_train['cp_type'] = data_train.cp_type.map({'trt_cp':0, 'ctl_vehicle':1})
df_test['cp_type'] = df_test.cp_type.map({'trt_cp':0, 'ctl_vehicle':1})

data_train['cp_dose'] = data_train.cp_dose.map({'D1':0, 'D2':1})
df_test['cp_dose'] = df_test.cp_dose.map({'D1':0, 'D2':1})

data_train.cp_type.value_counts()

In [ ]:
sns.set_style('whitegrid')
fig = plt.figure(figsize=(15,5))
#1 rows 2 cols
#first row, first col
ax1 = plt.subplot2grid((1,2),(0,0))
sns.countplot(uniques['count'], color='deepskyblue', alpha=0.75)
plt.title('Unique elements per drug [0,1]', fontsize=15, weight='bold')
#first row sec col
ax1 = plt.subplot2grid((1,2),(0,1))
sns.distplot(average['percentage'], color='orange', bins=20)
plt.title("The drugs mean distribution", fontsize=15, weight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(average_filtered['percentage'
                            ].sort_values(), average_filtered
            ['drug'], color=sns.color_palette('Reds',len(average_filtered)))
plt.title('Drugs - Higher presence in train samples', weight='bold', fontsize=15)
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.xlabel('Percentage', fontsize=13)
plt.show()

In [ ]:
inhibitors = [col for col in df_train_scored.columns if 'inhibitor' in col]
activators = [col for col in df_train_scored.columns if 'activator' in col]
antagonists = [col for col in df_train_scored.columns if 'antagonist' in col]
agonists = [col for col in df_train_scored.columns if 'agonist' in col]
modulators = [col for col in df_train_scored.columns if 'modulator' in col]
receptors = [col for col in df_train_scored.columns if 'receptor' in col]
receptors_ago = [col for col in df_train_scored.columns if 'receptor_agonist' in col]
receptors_anta = [col for col in df_train_scored.columns if 'receptor_antagonist' in col]


labelss= {'Drugs': ['inhibitors', 'activators', 'antagonists', 'agonists', 'receptors', 'receptors_ago', 'receptors_anta'],
          'Count':[112,5,32,60, 53, 24, 26]}


labels= pd.DataFrame(labelss)
labels=labels.sort_values(by=['Count'])
plt.figure(figsize=(15,5))
plt.bar(labels['Drugs'], labels['Count'], color=sns.color_palette('Reds',len(labels)))
plt.xticks(weight='bold')
plt.title('Compounds types', weight='bold', fontsize=15)
plt.show()

In [ ]:
ploth(drugs_tr)

In [ ]:
#Extract unique elements per column
cols2 = unscored.columns.to_list() # specify the columns whose unique values you want here
uniques2 = {col: unscored[col].nunique() for col in cols2}
uniques2=pd.DataFrame(uniques2, index=[0]).T
uniques2=uniques2.rename(columns={0:'count'})
uniques2= uniques2.drop('sig_id', axis=0)

#############################
### PLOT
plt.style.use('seaborn')
sns.set_style('whitegrid')
fig = plt.figure(figsize=(15,5))
#1 rows 2 cols
#first row, first col
ax1 = plt.subplot2grid((1,2),(0,0))
sns.countplot(uniques2['count'], palette='Blues', alpha=0.75)
plt.title('Un-scored: Unique elements per drug [0,1]', fontsize=13, weight='bold')
#first row sec col
ax1 = plt.subplot2grid((1,2),(0,1))
sns.countplot(uniques['count'], color='cyan', alpha=0.75)
plt.title('Scored: Unique elements per drug [0,1]', fontsize=13, weight='bold')
plt.show()

In [ ]:
print(f"{len(uniques2[uniques2['count']==1])} drugs without ANY mechanism of action in the nonscored dataset")

In [ ]:
average2=unscored.mean()
average2=pd.DataFrame(average2)
average2=average2.rename(columns={ 0: 'mean'})
average2['percentage']= average2['mean']*100
average_filtered2= average2[average2['mean'] > 0.01]
average_filtered2= average_filtered2.reset_index()
average_filtered2= average_filtered2.rename(columns={'index': 'drug'})

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = plt.subplot2grid((1,2),(0,0))
sns.distplot(average2['percentage'], color='blue', bins=20)
plt.title('Percentage of the nonscored MoAs in the samples',weight='bold', fontsize=13)
ax1 = plt.subplot2grid((1,2),(0,1))
sns.distplot(average['percentage'], color='gold', bins=20)
plt.title('Percentage of the scored MoAs in the samples',weight='bold', fontsize=13)
plt.show()

In [ ]:
corrs(adt, 'Drug', 'Drug 2', 15, thresh= 0.7)
corre= adt.corr()

In [ ]:
#Unstack the dataframe
s = corre.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
#Create new dataframe
so2= pd.DataFrame(so).reset_index()
so2= so2.rename(columns={0: 'correlation', 'level_0':'Drug 1', 'level_1': 'Drug2'})
#Filter out the coef 1 correlation between the same drugs
so2= so2[so2['correlation'] != 1]
#Drop pair duplicates
so2= so2.reset_index()
pos = [1,3,5,7,9, 11,13,15,17,19,21,23, 25, 27, 29, 31,33,35, 37, 39, 41, 43, 45]
so2= so2.drop(so2.index[pos])
#so2= so2.round(decimals=4)
so3=so2.head()
#Show the first 10 high correlations
cm = sns.light_palette("Red", as_cmap=True)
s = so2.head(16).style.background_gradient(cmap=cm)
s

In [ ]:
#High correlation adt 22 pairs
adt15= so2.head(22)
#Filter the drug names
adt_1=adt15['Drug 1'].values.tolist()
adt_2=adt15['Drug2'].values.tolist()
#Join the 2 lists
adt3= adt_1 + adt_2
#Keep unique elements and drop duplicates
adt4= list(dict.fromkeys(adt3))
#Filter out the selected drugs from the "all drugs treated" adt dataset
adt5= adt[adt4]
ploth(adt5)

In [ ]:
plt.style.use('seaborn')
sns.set_style('whitegrid')
fig = plt.figure(figsize=(15,5))
#1 rows 2 cols
#first row, first col
ax1 = plt.subplot2grid((1,2),(0,0))
sns.countplot(x='cp_type', data=df_test, palette='rainbow', alpha=0.75)
plt.title('Test: Control and treated samples', fontsize=15, weight='bold')
#first row sec col
ax1 = plt.subplot2grid((1,2),(0,1))
sns.countplot(x='cp_dose', data=df_test, palette='rainbow', alpha=0.75)
plt.title('Test: Treatment Doses: Low and High',weight='bold', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(13,3))
sns.distplot( df_train['cp_time'], color='gold', bins=5)
plt.title("Test: Treatment duration ", fontsize=15, weight='bold')
plt.show()

In [ ]:
#Filter out just the treated samples
treated2= df_test[df_test['cp_type']=='trt_cp']
treated_list2 = treated2['sig_id'].to_list()
full_tr= df_test[df_test['sig_id'].isin(treated_list2)]

#Select the columns c-
c_cols2 = [col for col in full_tr.columns if 'g-' in col]
#Filter the columns c-
cells2=treated2[c_cols2]
#Plot heatmap
plt.figure(figsize=(15,6))
sns.heatmap(cells2.corr(), cmap='coolwarm', alpha=0.9)
plt.title('Test: Correlation gene expression', fontsize=15, weight='bold')
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:
#Correlation between drugs
corre= cells2.corr()
#Unstack the dataframe
s = corre.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
#Create new dataframe
so2= pd.DataFrame(so).reset_index()
so2= so2.rename(columns={0: 'correlation', 'level_0':'Gene 1', 'level_1': 'Gene 2'})
#Filter out the coef 1 correlation between the same drugs
so2= so2[so2['correlation'] != 1]
#Drop pair duplicates
so2= so2.reset_index()
#so2= so2.sort_values(by=['correlation'])
pos = [1,3,5,7,9,11,13,15,17,19,21]
so2= so2.drop(so2.index[pos])
so2= so2.round(decimals=4)
so2=so2.drop('index', axis=1)
so4=so2.head(10)
cm = sns.light_palette("Red", as_cmap=True)
s = so2.head(10).style.background_gradient(cmap=cm)
s

In [ ]:
#Select the columns c-
c_cols3 = [col for col in df_test.columns if 'c-' in col]
#Filter the columns c-
cells3=treated[c_cols3]
#Plot heatmap
plt.figure(figsize=(12,6))
sns.heatmap(cells3.corr(), cmap='coolwarm', alpha=0.9)
plt.title('Correlation: Cell viability', fontsize=15, weight='bold')
plt.xticks(weight='bold')
plt.yticks(weight='bold')
plt.show()

In [ ]:

from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [ ]:
def build_model():
        
        inp = Input(shape=(X.shape[1], ))
        batch_norm = BatchNormalization()(inp)
        
        x1 = WeightNormalization(Dense(units=1024,
                                       activation='elu'))(batch_norm)
        drop = Dropout(0.5)(x1)
        batch_norm = BatchNormalization()(drop)
        
        
        x1 = WeightNormalization(Dense(640,
                                       activation='elu'))(batch_norm)
        drop = Dropout(0.5)(x1)
        batch_norm = BatchNormalization()(drop)
        
        x1 = WeightNormalization(Dense(768,
                                       activation='elu'))(batch_norm)
        drop = Dropout(0.5)(x1)
        batch_norm = BatchNormalization()(drop)        

        x1 = WeightNormalization(Dense(units=256,
                                       activation='elu'))(batch_norm)
        drop = Dropout(0.5)(x1)
        batch_norm = BatchNormalization()(drop)
        
        dense = Dense(Y.shape[1], activation='sigmoid')(batch_norm)
        
        model = Model(inputs=inp, outputs=dense)
        
        model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
        
        return model

In [ ]:
def get_log_loss(Y, val_Y):    
    full_loss = []
    for i, col in enumerate(Y.columns):
        loss = log_loss(Y.loc[:, col].values, val_Y[:, i].astype(float), labels=[0, 1])
        full_loss.append(loss)
    loss = np.mean(full_loss)
   
    return loss

In [ ]:
def train_model(X, Y, X_Test, folds = 3, seed=3, batch_size = 128, epochs = 50, learning_rate=1e-4):
    
    submission = sample_submission.drop('sig_id', axis=1).copy()
    submission.loc[:, :] = 0
    
    mkf = IterativeStratification(n_splits=folds, random_state=seed, shuffle=True)

    final_log_loss = []
    
    T_logloss = []
    V_logloss = []
    
    for n in range(seed):
        
        for fold, (train_idx, val_idx) in enumerate(mkf.split(X, Y)):

            print('\n Run - {}, Fold - {}'.format(n, fold))

            X_train = X.loc[train_idx, :]
            Y_train = Y.loc[train_idx, :]

            X_val = X.loc[val_idx, :]
            Y_val = Y.loc[val_idx, :]

            model = build_model()

            reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.15, patience=3, verbose=1,
                                               epsilon=learning_rate, mode='min')
            
            early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode= 'min')
            
            history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs,
                      callbacks=[reduce_lr_loss, early_stop], validation_data=(X_val, Y_val), verbose=2)

            Train_Pred = model.predict(X_train)
            Val_pred = model.predict(X_val)
        
            Train_Pred = Train_Pred/(seed*(fold+1))
            Val_pred = Val_pred/(seed*(fold+1))
            
            train_logloss = get_log_loss(Y_train, Train_Pred)
            val_logloss = get_log_loss(Y_val, Val_pred)
        
            print('Training Log Loss : {:.6}'.format(train_logloss))
            print('Validation Log Loss : {:.6}'.format(val_logloss))

            T_logloss.append(train_logloss)
            V_logloss.append(val_logloss)
            
            submission += model.predict(X_Test)
            
            submission = submission/((fold+1)*seed)
    
        final_T_logloss = np.mean(T_logloss)
        final_V_logloss = np.mean(V_logloss)

        print('Final Training Log Loss : {:.6}'.format(final_T_logloss))
        print('Final Validation Log Loss : {:.6}'.format(final_V_logloss))
    return submission

In [ ]:
data_train.sig_id

In [ ]:
X = data_train.drop('sig_id', axis=1)
Y = df_train_unscored.drop('sig_id', axis=1)

X_test = df_test.drop('sig_id', axis=1)

In [ ]:
xseed = 43
# number of folds for cv
nfolds = 5
# number of components to retain from PCA decomposition
nof_comp = 250
model_name = 'svm'

In [ ]:
# # create the submission file
# sub.iloc[:,1:] = Test_preds
# sub.to_csv('submission.csv', index=False)